### Data preparation

#### Import packages

In [1]:
# Install the libraries
import pandas as pd
import numpy as np
from pulp import *
import pulp.apis
from datetime import datetime
import time
import warnings
import yaml
from pandas.core.common import SettingWithCopyWarning
import json

In [2]:
solver_list = pulp.listSolvers(onlyAvailable=True)

In [3]:
# Ignore SettingWithCopyWarning 
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

In [4]:
def load_config():
    try:
        with open('config_file.yaml') as f:
            global constants
            constants = yaml.safe_load(f)
    except OSError as e:
        raise e
    except Exception as e:
        raise e

In [8]:
load_config()

#### Import data

In [9]:
# Import the data
# File name is specified in config file
data_input = pd.read_excel(constants['INPUT_FILE_NAME'], sheet_name='data', engine='pyxlsb')
real_input = pd.read_excel(constants['INPUT_FILE_NAME'], sheet_name='Real', engine='pyxlsb')
bandwidth_input = pd.read_excel(constants['INPUT_FILE_NAME'], sheet_name='PS', engine='pyxlsb')
match_input = pd.read_excel(constants['INPUT_FILE_NAME'], sheet_name='match', engine='pyxlsb')

In [10]:
# Copy datasets to implement further changes
data = data_input.copy()
real = real_input.copy()
bandwidth = bandwidth_input.copy()
match = match_input.copy()

#### Create dict with variable names

In [11]:
# Create a dict of names on a spreadsheets that we are going to use
names = {"Тариф" : "cost",
        "Плечо" : "distance",
        "Дата" : "date",
        "НБ" : "origin",
        "ОУ" : "point",
        "НП" : "brand",
        "НП_ПС" : "product",
        "Объем" : "volume",
        "reg": "region"} 

---

**Reorganize the data**

In [12]:
# Rename the names in dataframes
def rename_columns(df_list, names):
    for df in df_list:
         df.rename(columns=names, inplace=True)
    return df_list

In [13]:
data, real, bandwidth, match = rename_columns([data, real, bandwidth, match], names)

In [14]:
# Check for duplicated; will raise an error if there are duplicates
assert data[data.duplicated(subset = ['point', 'origin', 'region', 'brand', 'date'])].empty == True
assert real[real.duplicated(subset = ['point', 'brand', 'date'])].empty == True
assert bandwidth[bandwidth.duplicated(subset = ['origin', 'product', 'date'])].empty == True

In [15]:
# Make subset for the concrete time period
def select_time_period(df):
    df_subset = df[df['date'] == constants['MONTH']]
    return df_subset

# Make subset for the concrete region - ONLY FOR EXPERIMENTS, IN THE FINAL VERSION ALL REGIONS GO TOGETHER
def select_region(df):
    regions_to_select = []
    
    for region_number in constants['REGION']:
        region_name = 'Регион ' + str(region_number)
        regions_to_select.append(region_name)
        
    if 'region' in df.columns:
        df = df[(df['region'].isin(regions_to_select))]
    return df
    
# Adjust datatypes. Enumerate oilbases and petrol stations      
def adjust_datatypes(df_list):
    df_new = []
    number = 0
    oilbases_dict = dict()

    for i in bandwidth['origin'].unique():
        oilbases_dict[i] = number
        number+=1
        
    for df in df_list:
        if 'origin' in df.columns:
            df = df.replace({"origin": oilbases_dict})
        if 'point' in df.columns:
            df['point'] = df['point'].map(lambda x: int(x.strip('АЗС ')))
        df_new.append(df)
        
    return df_new

In [16]:
# Apply changes to dataframes
def apply_changes_dfs(data, real, bandwidth, match):
    
    data_subset = select_time_period(data)
    real_subset = select_time_period(real)
    bandwidth_subset = select_time_period(bandwidth)

    data_subset = select_region(data_subset)

    data_subset, real_subset, bandwidth_subset = adjust_datatypes([data_subset, real_subset, bandwidth_subset])
    data_subset['cost'] = data_subset['cost'].round(4)
    
    return data_subset, real_subset, bandwidth_subset

In [17]:
data_subset, real_subset, bandwidth_subset = apply_changes_dfs(data, real, bandwidth, match)

---

**Create dictionaries for brand-products matches**

In [18]:
# Create brand-product dict
def create_brand_product_dict(df):
    match_brand_product = dict(zip(df['brand'], df['product']))
    
    match_product_brand = dict()
    for key, value in match_brand_product.items():
        match_product_brand.setdefault(value, list()).append(key)
    return match_brand_product, match_product_brand

# Create numeration dict and inverted dict
def create_numeration_dict(df, parameter):
    match_dict = dict(enumerate(df[parameter]))
    inv_dict = {v: k for k, v in match_dict.items()}
    return match_dict, inv_dict

# Create numeration dict and inverted dict
def create_prod_numeration_dict(df, parameter):
    match_dict = dict(enumerate(df[parameter].unique()))
    inv_dict = {v: k for k, v in match_dict.items()}
    return match_dict, inv_dict

In [19]:
# Create 6 dictionaries (brand-product, brand-number, product-number, and inverted)
match_brand_product, match_product_brand = create_brand_product_dict(match)
match_brand_number, match_number_brand = create_numeration_dict(match,'brand')
match_product_number, match_number_product = create_prod_numeration_dict(match,'product')

---

In [20]:
# Create name of variable
def create_var_Name(row):
    var_name = 'P' + str(int(row["point"])) + 'O' + str(int(row["origin"])) +  '_brand_' + str(int(row["brand_number"]))
    return var_name

# Reorganize the data. Select only specified columns, enumerate brands and products
def reorganize_df(df, columns_to_keep, match_number_brand, match_number_product):
    df_reorganized = df.reset_index().loc[:, columns_to_keep]
    df_reorganized['brand_number'] = df_reorganized['brand'].map(lambda x: match_number_brand.get(x))
    df_reorganized['product_number'] = df_reorganized['product'].map(lambda x: match_number_product.get(x))
    
    return df_reorganized

In [21]:
columns_to_keep = ['origin', 'point', 'brand', 'date', 'product', 'cost', 'region']
df_reorganized = reorganize_df(data_subset, columns_to_keep, match_number_brand, match_number_product)

# Create a name variable
df_reorganized['var_Name'] = df_reorganized.apply(lambda row: create_var_Name(row), axis=1)  

---

**Create demand and supply dataframes**

In [22]:
# Create demand and supply dataframes
# Demand - volume of each product needed to be transported to the point
# Supply - volume of each product available on the origin

def create_demand_supply_dfs(real, bandwidth):
    demand = real[['point', 'brand', 'product', 'volume', 'date']]
    demand.rename(columns={'volume':'demand'}, inplace=True)
    demand['brand_number'] = demand['brand'].map(lambda x: match_number_brand.get(x))
    demand['product_number'] = demand['product'].map(lambda x: match_number_product.get(x))
    demand['demand'] = demand['demand'].round(4)
    
    supply = bandwidth[['origin', 'product', 'volume', 'date']]
    supply.rename(columns={'volume':'supply'}, inplace=True)
    supply['product_number'] = supply['product'].map(lambda x: match_number_product.get(x))
    supply['supply'] = supply['supply'].round(4)
    
    return demand, supply

In [23]:
demand, supply = create_demand_supply_dfs(real_subset, bandwidth_subset)

In [24]:
# Limit demand and supply with only needed origins and points
supply = supply[supply['origin'].isin(df_reorganized.origin.unique())]
demand = demand[demand['point'].isin(df_reorganized.point.unique())]

In [25]:
# Make a copy
df_reorganized_trans = df_reorganized.copy()
print('Length of original df_reorganized: ', len(df_reorganized))

# Check for where ALL volumes equals == 0 
df_trans= supply[supply['supply'] <= 0][['origin', 'product_number']]
origins_zero_volume = set(zip(df_trans.origin, df_trans.product_number))

pairs_origins_products_to_delete = []
pairs_points_brands_to_delete = []

# Filter df_reogranized
for _, row in df_reorganized.iterrows():

    origin, point, product, brand = row['origin'], row['point'], row['product_number'], row['brand_number']
    
    if supply[(supply['origin'] == origin) & (supply['product_number'] == product)].empty:
        pairs_origins_products_to_delete.append((origin, product))

    if demand[(demand['point'] == point) & (demand['brand_number'] == brand)].empty:
        pairs_points_brands_to_delete.append((point, brand))

pairs_origins_products_to_delete = set(pairs_origins_products_to_delete)
pairs_points_brands_to_delete = set(pairs_points_brands_to_delete)

# Update origins to delete
pairs_origins_products_to_delete.update(origins_zero_volume)

df_reorganized_trans['origin_product'] = list(zip(df_reorganized_trans.origin, df_reorganized_trans.product_number))
df_reorganized_trans['point_brand'] = list(zip(df_reorganized_trans.point, df_reorganized_trans.brand_number))
  
df_reorganized_trans = df_reorganized_trans.drop(df_reorganized_trans[df_reorganized_trans['origin_product'].isin(pairs_origins_products_to_delete)].index)
df_reorganized_trans = df_reorganized_trans.drop(df_reorganized_trans[df_reorganized_trans['point_brand'].isin(pairs_points_brands_to_delete)].index)
   
        
# Reassign df
df_reorganized = df_reorganized_trans
df_reorganized = df_reorganized.reset_index().drop('index', axis=1)
print('Length of new df_reorganized: ', len(df_reorganized))

print('\nNumber of pairs of origins-product dropped: ', len(pairs_origins_products_to_delete))
print('Number of pairs of points-brands: ', len(pairs_points_brands_to_delete))

Length of original df_reorganized:  61877
Length of new df_reorganized:  53802

Number of pairs of origins-product dropped:  33
Number of pairs of points-brands:  0


### Usage of puLP

#### Define function

In [26]:
brands = df_reorganized['brand_number'].unique()

In [27]:
brand_info_map = {}

def make_brand(brand_id, name):
    return {'name': name, 'id': brand_id, 'product_transportations': {}}

In [28]:
def make_transportation(origin, point, cost, var_name):
    return {'origin': origin, 'point': point, 'cost': cost, 'var_name': var_name}

In [29]:
def add_transportation_to_brand(brand, product_id, transportation):
    if brand['product_transportations'].get(product_id) == None:
        brand['product_transportations'][product_id] = [transportation]
        return brand
    
    brand['product_transportations'].get(product_id).append(transportation)  

    return brand

In [30]:
def append_row_to_map(brand_map, row):
    brand_id = row['brand_number']
    
    if brand_map.get(brand_id) == None:
        brand_map[brand_id] = make_brand(brand_id, row['brand'])
        
    brand_item = brand_map.get(brand_id)
    product_id = row['product_number']
    
    transportation = make_transportation(row['origin'], row['point'], row['cost'], row['var_Name'])
    brand_map[brand_id] = add_transportation_to_brand(brand_item, product_id, transportation)
    
    return brand_map

for i, row in df_reorganized.iterrows():
    brand_info_map = append_row_to_map(brand_info_map, row)

In [31]:
# Make dicts for groups - either D (diesel) or P (petrol)
def make_match_dicts(match_brand_product):
    
    match_brand_group = dict()
    match_product_group = dict()

    for k,v in match_brand_product.items():
        if "ДТ" in v:
            match_brand_group[k] = "D"
            match_product_group[v] = "D"
        
        elif ("АИ" in v) | ("Аи" in v):
            match_brand_group[k] = "P"
            match_product_group[v] = "P"

    match_group_brand = dict()

    for k,v in match_brand_group.items():
        match_group_brand.setdefault(v, list()).append(k)
    
    
    match_group_product = dict()

    for k,v in match_product_group.items():
        match_group_product.setdefault(v, list()).append(k)
    
    return match_brand_group, match_product_group, match_group_brand, match_group_product

In [32]:
match_brand_group, match_product_group, match_group_brand, match_group_product = make_match_dicts(match_brand_product)

In [33]:
# Function for dividing supply between different products
def merge_supply_by_product(brand_number, match_brand_product, match_brand_number, match_product_brand, match_number_brand):
    """This function returns a set of brands' numbers that are produced from the same product"""
    brand_name = match_brand_number[brand_number]
    product_name = match_brand_product[brand_name]
    brands_overlap_names = match_product_brand[product_name]
    brands_overlap_numbers = set(match_number_brand.get(item) for item in brands_overlap_names)

    return brands_overlap_numbers

In [34]:
# Function for dividing supply between different groups
def merge_supply_by_group(brand_number, match_brand_number, match_number_brand, match_brand_group, match_group_brand):
    """This function returns a set of brands' numbers that are produced from the same product group - P or D"""
    brand_name = match_brand_number[brand_number]
    group_name = match_brand_group[brand_name]
    brands_overlap_names = match_group_brand[group_name]
    brands_overlap_numbers = set(match_number_brand.get(item) for item in brands_overlap_names)

    return brands_overlap_numbers

In [35]:
# Function for dividing supply between different brands
def merge_supply(brand_number, match_brand_product, match_brand_number, match_product_brand, match_number_brand):
    """This function returns a set of brands' numbers that are produced from the same product"""
    brand_name = match_brand_number[brand_number]
    product_name = match_brand_product[brand_name]
    brands_overlap_names = match_product_brand[product_name]
    brands_overlap_numbers = set(match_number_brand.get(item) for item in brands_overlap_names)

    return brands_overlap_numbers

In [36]:
# Get the list of unique points inside each brand
def extract_unique_points_of_brand(brand):
    points = []
    for product_id in brand.get('product_transportations').keys():
        for transportation in brand.get('product_transportations').get(product_id):
            points.append(transportation.get('point'))

    points = list(set(points))
    
    return points

# Get the list of unique origins inside each brand
def extract_unique_origins_of_brand(brand):
    origins = []
    for transportations in brand.get('product_transportations').values():
        for transportation in transportations:
            origins.append(transportation.get('origin'))

    origins = list(set(origins))
    
    return origins

# Extract the transportation of products from map dictionary
def get_brand_transportations(brand):
    brand_transpotrations = []
    for product_id in brand.get('product_transportations').keys():
        for transportation in brand.get('product_transportations').get(product_id):
            brand_transpotrations.append(transportation)
                
    return brand_transpotrations

# Get the list of unique origins (overall)
def extract_unique_origins(brand_map):
    origins = []
    for brand in brand_map.values():
        origins = origins + extract_unique_origins_of_brand(brand)

    return list(set(origins))

# Get the list of unique points (overall)
def extract_unique_points(brand_map):
    points = []
    for brand in brand_map.values():
        points = points + extract_unique_points_of_brand(brand)
        
    return list(set(points))

# Get transportations of concrete brand
def get_transportations(brand_map):
    transportations = []
    
    for brand in brand_map.values():
        transportations = transportations + get_brand_transportations(brand)
        
    return transportations

In [37]:
def define_problem(df):
    x = [pulp.LpVariable(df['var_Name'][i], lowBound = 0, cat='Continuous') for i in np.arange(0, len(df))]
    # Set function's goal
    problem = pulp.LpProblem('0', sense=LpMinimize)
    # Create a function
    problem += pulp.LpAffineExpression([(x[i], df['cost'][i]) for i in np.arange(0, len(df))])
    x_map = {}
    
    for i in x:
        x_map[str(i)] = i
    
    return problem, x, x_map

In [38]:
# Make restriction on the demand (concrete volume of product needed for point)
def make_restriction_demand(record, problem, demand, x, brands):
    n_constraint = 1
    
    for brand_id in brands:
        brand = record.get(brand_id);
        points = extract_unique_points_of_brand(brand)
    
        for p in points:
            if p in list(real_subset.point.unique()):
                transportations = get_brand_transportations(brand)
                brand_point_transpotrations = [transportation for transportation in transportations if transportation.get('point') == p]
                const = float(demand[(demand['point'] == p) & (demand['brand_number'] == brand_id)]['demand'])
                # Add a restriction on demand == const
                problem+= pulp.LpAffineExpression([(x.get(transportation.get('var_name')), 1) for transportation in brand_point_transpotrations ]) == const, str(n_constraint)
                n_constraint +=1
        
    return problem, n_constraint

In [39]:
# Make restriction on the supply (concrete volume of product available on origin)
# Limitations by month, and by group S, D - upper bound
def make_restriction_supply_by_group_upper(problem, record, bandwidth, supply, x, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, match_product_group, match_group_product, match_group_brand, match_brand_group, n_constraint):
    # Set for storing already processed brands 
    brands_accounted_for = set()
    
    # Store origin with infinite supply in a separate list
    #origins_with_inf_supply = list(bandwidth[bandwidth['volume'] == np.inf]['origin'].unique())
    
    for b in brands:
        if b in brands_accounted_for:
            continue
        
        brands_overlap_numbers = merge_supply_by_group(b, match_brand_number, match_number_brand, match_brand_group, match_group_brand)
        brands_accounted_for.update(brands_overlap_numbers)

        brands_list = [brand for brand in record.values() if brand.get('id') in brands_overlap_numbers]

        origins = []
        for brand in brands_list:
            origins += extract_unique_origins_of_brand(brand)
            
        origins = list(set(origins))
        
        transportations_list = []
        for brand in brands_list:
            transportations_list += get_brand_transportations(brand)

        for o in origins:
            
            origin_transportations = [transportation for transportation in transportations_list if transportation.get('origin') == o]
            product_name = match_brand_product[match_brand_number[b]]
            group_name = match_product_group[product_name]
            products_share_group = match_group_product[group_name]
            const = float(supply[(supply['origin'] == o) & (supply['product'].isin(products_share_group))]['supply'].sum())
            
            # Add a restriction on supply <= upper_const
            problem+= pulp.LpAffineExpression([(x.get(i.get('var_name')), 1) for i in origin_transportations ]) <= const, str(n_constraint)
            n_constraint +=1   
                
    return problem, n_constraint

In [40]:
# Make restriction on supply (restriction on lower percentage of loading)
# Limitation by month, group D,and S - only lower bound
def make_restriction_supply_by_group_lower(problem, record, bandwidth, supply, x, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, match_product_group, match_group_product, match_group_brand, match_brand_group, n_constraint):
    # Set for storing already processed brands 
    brands_accounted_for = set()
    
    # Store origin with infinite supply in a separate list
    number = 0
    oilbases_dict = dict()

    for i in bandwidth['origin'].unique():
        oilbases_dict[i] = number
        number+=1
        
    origins_with_inf_supply = set(oilbases_dict.values()).difference(constants['ORIGINS_TO_CONTROL'])
    
    for b in brands:
        if b in brands_accounted_for:
            continue
        
        brands_overlap_numbers = merge_supply_by_group(b, match_brand_number, match_number_brand, match_brand_group, match_group_brand)
        brands_accounted_for.update(brands_overlap_numbers)

        brands_list = [brand for brand in record.values() if brand.get('id') in brands_overlap_numbers]

        origins = []
        for brand in brands_list:
            origins += extract_unique_origins_of_brand(brand)
            
        origins = list(set(origins))
        
        transportations_list = []
        for brand in brands_list:
            transportations_list += get_brand_transportations(brand)

        for o in origins:
            
            origin_transportations = [transportation for transportation in transportations_list if transportation.get('origin') == o]

            product_name = match_brand_product[match_brand_number[b]]
            group_name = match_product_group[product_name]
            products_share_group = match_group_product[group_name]
            const = float(supply[(supply['origin'] == o) & (supply['product'].isin(products_share_group))]['supply'].sum())            
            
            if o in origins_with_inf_supply:
                continue       

            else:

                # Add a restriction on supply >= lower_const
                const *= constants['PERCENT_LOADING']
                problem+= pulp.LpAffineExpression([(x.get(i.get('var_name')), 1) for i in origin_transportations ]) >= const, str(n_constraint)
                n_constraint +=1
                    
    return problem, n_constraint

In [41]:
# Limitation by month and product - upper bound
def make_restriction_supply_by_product_upper(problem, record, bandwidth, supply, x, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, n_constraint):
    # Set for storing already processed brands 
    brands_accounted_for = set()
    
    # Store origin with infinite supply in a separate list
    #origins_with_inf_supply = list(bandwidth[bandwidth['volume'] == np.inf]['origin'].unique())
    
    for b in brands:
        if b in brands_accounted_for:
            continue
        
        brands_overlap_numbers = merge_supply_by_product(b, match_brand_product, match_brand_number, match_product_brand, match_number_brand)
        brands_accounted_for.update(brands_overlap_numbers)

        brands_list = [brand for brand in record.values() if brand.get('id') in brands_overlap_numbers]

        origins = []
        for brand in brands_list:
            origins += extract_unique_origins_of_brand(brand)
            
        origins = list(set(origins))
        
        transportations_list = []
        for brand in brands_list:
            transportations_list += get_brand_transportations(brand)
            
            
        for o in origins:
            
            origin_transportations = [transportation for transportation in transportations_list if transportation.get('origin') == o]
            product_name = match_brand_product[match_brand_number[b]]
            const = float(supply[(supply['origin'] == o) & (supply['product'] == product_name)]['supply'])
            
            # Add a restriction on supply <= upper_const
            problem+= pulp.LpAffineExpression([(x.get(i.get('var_name')), 1) for i in origin_transportations ]) <= const, str(n_constraint)
            n_constraint +=1

    return problem, n_constraint

In [42]:
# Limitation by month and product - lower bound
def make_restriction_supply_by_product_lower(problem, record, bandwidth, supply, x, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, n_constraint):
    # Set for storing already processed brands 
    brands_accounted_for = set()
    
    # Store origin with infinite supply in a separate list
    number = 0
    oilbases_dict = dict()

    for i in bandwidth['origin'].unique():
        oilbases_dict[i] = number
        number+=1
        
    origins_with_inf_supply = set(oilbases_dict.values()).difference(constants['ORIGINS_TO_CONTROL'])
    
    for b in brands:
        if b in brands_accounted_for:
            continue
        
        brands_overlap_numbers = merge_supply_by_product(b, match_brand_product, match_brand_number, match_product_brand, match_number_brand)
        brands_accounted_for.update(brands_overlap_numbers)

        brands_list = [brand for brand in record.values() if brand.get('id') in brands_overlap_numbers]

        origins = []
        for brand in brands_list:
            origins += extract_unique_origins_of_brand(brand)
            
        origins = list(set(origins))
        
        transportations_list = []
        for brand in brands_list:
            transportations_list += get_brand_transportations(brand)

        for o in origins:
            
            origin_transportations = [transportation for transportation in transportations_list if transportation.get('origin') == o]
            product_name = match_brand_product[match_brand_number[b]]
            const = float(supply[(supply['origin'] == o) & (supply['product'] == product_name)]['supply'])
            
            if o in origins_with_inf_supply:
                continue
                
            else:

                # Add a restriction on supply >= lower_const
                const *= constants['PERCENT_LOADING']
                problem+= pulp.LpAffineExpression([(x.get(i.get('var_name')), 1) for i in origin_transportations ]) >= const, str(n_constraint)
                n_constraint +=1 

    return problem, n_constraint

In [43]:
# Make restriction on supply by sum of all products - lower bound
def make_restriction_supply_by_all_products_lower(problem, record, supply, x, brands, n_constraint):
    # Set for storing already processed brands 
    # brands_accounted_for = set()
    
    # Store origin with infinite supply in a separate list
    number = 0
    oilbases_dict = dict()

    for i in bandwidth['origin'].unique():
        oilbases_dict[i] = number
        number+=1
        
    origins_with_inf_supply = set(oilbases_dict.values()).difference(constants['ORIGINS_TO_CONTROL'])
    
    brands_list = [brand for brand in record.values()]
    origins = []
    
    for brand in brands_list:
        origins += extract_unique_origins_of_brand(brand)
            
    origins = list(set(origins))
        
    for o in origins:
        if o in origins_with_inf_supply:
            continue

        else:
            transportations_list = []
            for brand in brands_list:
                transportations_list += get_brand_transportations(brand)            
            
                            
            origin_transportations = [transportation for transportation in transportations_list if transportation.get('origin') == o]

            const = float(supply[(supply['origin'] == o)]['supply'].sum())
            const *= constants['PERCENT_LOADING']
            
            # Add a restriction on supply >= lower_const
            problem+= pulp.LpAffineExpression([(x.get(i.get('var_name')), 1) for i in origin_transportations ]) >= const, str(n_constraint)
            n_constraint +=1
    
    return problem, n_constraint

In [44]:
# Function for creating dummies names
def create_dummy_Name(df):
    dummies = {}
    
    # Create a name variable
    dummies_names = set(df.apply(lambda row: 'y_P' + str(int(row["point"])) + 'O' + str(int(row["origin"])), axis=1))

    # Create dummy variables and store them in a dict
    for name in dummies_names:
        dummies[name] = pulp.LpVariable(name, lowBound=0, upBound = 1, cat = 'Integer')
        
    return dummies

In [45]:
# Create points - origins dictionary
def create_points_dict(df):
    points_dict = {}
    for p in list(df['point'].unique()):
        origins_available = list(df[df['point'] == p]['origin'].unique())
        if points_dict.get(p) == None:
            points_dict[p] = origins_available
    return points_dict

points_dict = create_points_dict(df_reorganized)

In [46]:
# Make restriction on dummy variables (all products are delivered from the same origin to the same point)
def make_restriction_dummy(record, problem, n_constraint, dummies, x, points_dict):
    
    transportations = get_transportations(record)
    
    for p in points_dict.keys():
        
        trans = []
        
        for o in points_dict[p]:
            
            po_transportations = [transportation for transportation in transportations if transportation.get('origin') == o and transportation.get('point') == p]
            
            string = "y_P" + str(p) + "O" + str(o)
            d = dummies.get(string)
            
            # Restriction (...) <= M * Y_PnOm
            problem+= pulp.LpAffineExpression([(x.get(i.get('var_name')), 1) for i in po_transportations ]) <=  constants['M'] * d, str(n_constraint)
            n_constraint += 1
            trans.append(d)
            
        # Restriction (y_PnOm1 + y_PnOm2 + ...) == 1
        problem += pulp.LpAffineExpression([(i, 1) for i in trans]) == 1, str(n_constraint)
        n_constraint += 1  
            
    return problem, n_constraint

### SCIP

In [47]:
scip_start = time.time()

dummies = create_dummy_Name(df_reorganized)

problem_defined, x, x_map = define_problem(df_reorganized)

problem, n_constraint = make_restriction_demand(brand_info_map, problem_defined, demand, x_map, brands)

if constants['UPPER_BOUND'] == 'group':
    problem, n_constraint, = make_restriction_supply_by_group_upper(problem, brand_info_map, bandwidth_subset, supply, x_map, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, match_product_group, match_group_product, match_group_brand, match_brand_group, n_constraint)
elif constants['UPPER_BOUND'] == 'product':
    problem, n_constraint = make_restriction_supply_by_product_upper(problem, brand_info_map, bandwidth_subset, supply, x_map, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, n_constraint)

if constants['LOWER_BOUND'] == 'group':
    problem, n_constraint = make_restriction_supply_by_group_lower(problem, brand_info_map, bandwidth_subset, supply, x_map, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, match_product_group, match_group_product, match_group_brand, match_brand_group, n_constraint)
elif constants['LOWER_BOUND'] == 'product':
    problem, n_constraint = make_restriction_supply_by_product_lower(problem, brand_info_map, bandwidth_subset, supply, x, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, n_constraint)
elif constants['LOWER_BOUND'] == 'all products':
    problem, n_constraint = make_restriction_supply_by_all_products_lower(problem, brand_info_map, supply, x_map, brands, n_constraint)
    
problem, n_constraint = make_restriction_dummy(brand_info_map, problem, n_constraint, dummies, x_map, points_dict)

scip_end = time.time()
print('Время работы SCIP: ', scip_end-scip_start)

Время работы SCIP:  319.4808301925659


In [48]:
# def main(problem):
#     start = time.time()
#     status = problem.solve(SCIP_CMD())
    
#     print(status)
#     # Print the results
#     print ("Результат:")
#     for variable in problem.variables():
#         print (variable.name, "=", variable.varValue)
#     print ("Стоимость доставки:")
#     print (abs(problem.objective.value()))
#     stop = time.time()
#     print ("Время :")
#     print(stop - start)

In [49]:
# # This is SCIP_CMD
# if __name__ == '__main__':
#     main(problem)
#     scip_end = time.time()
#     print('Время работы SCIP: ', scip_end)

### CBC

In [50]:
cbc_start = time.time()

dummies = create_dummy_Name(df_reorganized)

problem_defined, x, x_map = define_problem(df_reorganized)

problem, n_constraint = make_restriction_demand(brand_info_map, problem_defined, demand, x_map, brands)

if constants['UPPER_BOUND'] == 'group':
    problem, n_constraint, = make_restriction_supply_by_group_upper(problem, brand_info_map, bandwidth_subset, supply, x_map, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, match_product_group, match_group_product, match_group_brand, match_brand_group, n_constraint)
elif constants['UPPER_BOUND'] == 'product':
    problem, n_constraint = make_restriction_supply_by_product_upper(problem, brand_info_map, bandwidth_subset, supply, x_map, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, n_constraint)

if constants['LOWER_BOUND'] == 'group':
    problem, n_constraint = make_restriction_supply_by_group_lower(problem, brand_info_map, bandwidth_subset, supply, x_map, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, match_product_group, match_group_product, match_group_brand, match_brand_group, n_constraint)
elif constants['LOWER_BOUND'] == 'product':
    problem, n_constraint = make_restriction_supply_by_product_lower(problem, brand_info_map, bandwidth_subset, supply, x, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, n_constraint)
elif constants['LOWER_BOUND'] == 'all products':
    problem, n_constraint = make_restriction_supply_by_all_products_lower(problem, brand_info_map, supply, x_map, brands, n_constraint)
    
problem, n_constraint = make_restriction_dummy(brand_info_map, problem, n_constraint, dummies, x_map, points_dict)

cbc_end = time.time()
print('Время введения ограничений: ', cbc_end-cbc_start)

Время введения ограничений:  335.5596318244934


In [51]:
def main(problem):
    start = time.time()
    status = problem.solve(PULP_CBC_CMD(timeLimit=60*20, warmStart=True))
    
    print(status)
    # Print the results
    print ("Результат:")
    for variable in problem.variables():
        print (variable.name, "=", variable.varValue)
    print ("Стоимость доставки:")
    print (abs(problem.objective.value()))
    stop = time.time()
    print ("Время :")
    print(stop - start)

In [52]:
# This is PULP_CBC_CMD
if __name__ == '__main__':
    main(problem)
    cbc_end = time.time()
    print('Время работы солвера CBC: ', cbc_end-cbc_start)

1
Результат:
P1000O0_brand_1 = 0.0
P1000O0_brand_2 = 0.0
P1000O0_brand_3 = 0.0
P1000O0_brand_5 = 0.0
P1000O16_brand_1 = 0.0
P1000O16_brand_2 = 0.0
P1000O16_brand_3 = 0.0
P1000O16_brand_5 = 0.0
P1000O1_brand_1 = 0.0
P1000O1_brand_2 = 0.0
P1000O1_brand_3 = 0.0
P1000O1_brand_5 = 0.0
P1000O21_brand_1 = 0.0
P1000O21_brand_2 = 0.0
P1000O21_brand_3 = 0.0
P1000O21_brand_5 = 0.0
P1000O22_brand_1 = 0.0
P1000O22_brand_2 = 0.0
P1000O22_brand_3 = 0.0
P1000O22_brand_5 = 0.0
P1000O23_brand_1 = 130.6236
P1000O23_brand_2 = 104.6599
P1000O23_brand_3 = 24.258
P1000O23_brand_5 = 65.4264
P1000O24_brand_1 = 0.0
P1000O24_brand_2 = 0.0
P1000O24_brand_3 = 0.0
P1000O24_brand_5 = 0.0
P1000O27_brand_1 = 0.0
P1000O27_brand_2 = 0.0
P1000O27_brand_3 = 0.0
P1000O27_brand_5 = 0.0
P1000O30_brand_1 = 0.0
P1000O30_brand_2 = 0.0
P1000O30_brand_3 = 0.0
P1000O30_brand_5 = 0.0
P1000O32_brand_1 = 0.0
P1000O32_brand_5 = 0.0
P1000O33_brand_1 = 0.0
P1000O33_brand_2 = 0.0
P1000O33_brand_3 = 0.0
P1000O33_brand_5 = 0.0
P1000O34_bra

P1006O32_brand_1 = 0.0
P1006O32_brand_5 = 0.0
P1006O33_brand_1 = 0.0
P1006O33_brand_3 = 0.0
P1006O33_brand_5 = 0.0
P1006O34_brand_0 = 0.0
P1006O34_brand_1 = 0.0
P1006O34_brand_3 = 0.0
P1006O34_brand_5 = 0.0
P1006O35_brand_1 = 0.0
P1006O35_brand_3 = 0.0
P1006O35_brand_5 = 0.0
P1006O37_brand_1 = 0.0
P1006O37_brand_3 = 0.0
P1006O37_brand_5 = 0.0
P1006O38_brand_0 = 0.0
P1006O38_brand_1 = 0.0
P1006O38_brand_3 = 0.0
P1006O38_brand_5 = 0.0
P1006O39_brand_1 = 0.0
P1006O39_brand_3 = 0.0
P1006O39_brand_5 = 0.0
P1006O41_brand_0 = 0.0
P1006O41_brand_1 = 0.0
P1006O41_brand_3 = 0.0
P1006O41_brand_5 = 0.0
P1006O4_brand_1 = 0.0
P1006O4_brand_3 = 0.0
P1006O4_brand_5 = 0.0
P1006O5_brand_0 = 0.0
P1006O5_brand_1 = 0.0
P1006O5_brand_3 = 0.0
P1006O5_brand_5 = 0.0
P1006O6_brand_0 = 0.0
P1006O6_brand_1 = 0.0
P1006O6_brand_3 = 0.0
P1006O6_brand_5 = 0.0
P1006O7_brand_0 = 0.0
P1006O7_brand_1 = 0.0
P1006O7_brand_3 = 0.0
P1006O7_brand_5 = 0.0
P1006O8_brand_0 = 0.0
P1006O8_brand_1 = 0.0
P1006O8_brand_3 = 0.0
P1006O

P1012O30_brand_1 = 0.0
P1012O30_brand_3 = 0.0
P1012O30_brand_5 = 0.0
P1012O32_brand_1 = 0.0
P1012O32_brand_5 = 0.0
P1012O33_brand_1 = 0.0
P1012O33_brand_3 = 0.0
P1012O33_brand_5 = 0.0
P1012O34_brand_1 = 0.0
P1012O34_brand_3 = 0.0
P1012O34_brand_5 = 0.0
P1012O35_brand_1 = 0.0
P1012O35_brand_3 = 0.0
P1012O35_brand_5 = 0.0
P1012O37_brand_1 = 0.0
P1012O37_brand_3 = 0.0
P1012O37_brand_5 = 0.0
P1012O38_brand_1 = 0.0
P1012O38_brand_3 = 0.0
P1012O38_brand_5 = 0.0
P1012O39_brand_1 = 0.0
P1012O39_brand_3 = 0.0
P1012O39_brand_5 = 0.0
P1012O41_brand_1 = 0.0
P1012O41_brand_3 = 0.0
P1012O41_brand_5 = 0.0
P1012O4_brand_1 = 0.0
P1012O4_brand_3 = 0.0
P1012O4_brand_5 = 0.0
P1012O5_brand_1 = 0.0
P1012O5_brand_3 = 0.0
P1012O5_brand_5 = 0.0
P1012O6_brand_1 = 0.0
P1012O6_brand_3 = 0.0
P1012O6_brand_5 = 0.0
P1012O7_brand_1 = 0.0
P1012O7_brand_3 = 0.0
P1012O7_brand_5 = 0.0
P1012O8_brand_1 = 0.0
P1012O8_brand_3 = 0.0
P1012O8_brand_5 = 0.0
P1013O0_brand_1 = 0.0
P1013O0_brand_2 = 0.0
P1013O0_brand_3 = 0.0
P1013O

P1018O37_brand_1 = 0.0
P1018O37_brand_2 = 0.0
P1018O37_brand_3 = 0.0
P1018O37_brand_5 = 0.0
P1018O38_brand_1 = 0.0
P1018O38_brand_2 = 0.0
P1018O38_brand_3 = 0.0
P1018O38_brand_5 = 0.0
P1018O39_brand_1 = 0.0
P1018O39_brand_2 = 0.0
P1018O39_brand_3 = 0.0
P1018O39_brand_5 = 0.0
P1018O41_brand_1 = 0.0
P1018O41_brand_2 = 0.0
P1018O41_brand_3 = 0.0
P1018O41_brand_5 = 0.0
P1018O4_brand_1 = 0.0
P1018O4_brand_2 = 0.0
P1018O4_brand_3 = 0.0
P1018O4_brand_5 = 0.0
P1018O5_brand_1 = 0.0
P1018O5_brand_2 = 0.0
P1018O5_brand_3 = 0.0
P1018O5_brand_5 = 0.0
P1018O6_brand_1 = 0.0
P1018O6_brand_2 = 0.0
P1018O6_brand_3 = 0.0
P1018O6_brand_5 = 0.0
P1018O7_brand_1 = 0.0
P1018O7_brand_2 = 0.0
P1018O7_brand_3 = 0.0
P1018O7_brand_5 = 0.0
P1018O8_brand_1 = 0.0
P1018O8_brand_2 = 0.0
P1018O8_brand_3 = 0.0
P1018O8_brand_5 = 0.0
P1019O0_brand_1 = 0.0
P1019O0_brand_2 = 0.0
P1019O0_brand_3 = 0.0
P1019O0_brand_5 = 0.0
P1019O16_brand_1 = 0.0
P1019O16_brand_2 = 0.0
P1019O16_brand_3 = 0.0
P1019O16_brand_5 = 0.0
P1019O1_bran

P1024O0_brand_5 = 0.0
P1024O16_brand_1 = 0.0
P1024O16_brand_2 = 0.0
P1024O16_brand_3 = 0.0
P1024O16_brand_5 = 0.0
P1024O1_brand_1 = 0.0
P1024O1_brand_2 = 0.0
P1024O1_brand_3 = 0.0
P1024O1_brand_5 = 0.0
P1024O21_brand_1 = 0.0
P1024O21_brand_2 = 0.0
P1024O21_brand_3 = 0.0
P1024O21_brand_5 = 0.0
P1024O22_brand_1 = 0.0
P1024O22_brand_2 = 0.0
P1024O22_brand_3 = 0.0
P1024O22_brand_5 = 0.0
P1024O23_brand_1 = 0.0
P1024O23_brand_2 = 0.0
P1024O23_brand_3 = 0.0
P1024O23_brand_5 = 0.0
P1024O24_brand_1 = 0.0
P1024O24_brand_2 = 0.0
P1024O24_brand_3 = 0.0
P1024O24_brand_5 = 0.0
P1024O27_brand_1 = 0.0
P1024O27_brand_2 = 0.0
P1024O27_brand_3 = 0.0
P1024O27_brand_5 = 0.0
P1024O30_brand_1 = 0.0
P1024O30_brand_2 = 0.0
P1024O30_brand_3 = 0.0
P1024O30_brand_5 = 0.0
P1024O32_brand_1 = 0.0
P1024O32_brand_5 = 0.0
P1024O33_brand_1 = 0.0
P1024O33_brand_2 = 0.0
P1024O33_brand_3 = 0.0
P1024O33_brand_5 = 0.0
P1024O34_brand_1 = 0.0
P1024O34_brand_2 = 0.0
P1024O34_brand_3 = 0.0
P1024O34_brand_5 = 0.0
P1024O35_brand_1

...

y_P742O3 = 0.0
y_P742O30 = 0.0
y_P742O33 = 0.0
y_P742O7 = 0.0
y_P742O8 = 0.0
y_P743O1 = 0.0
y_P743O11 = 0.0
y_P743O14 = 0.0
y_P743O16 = 0.0
y_P743O18 = 0.0
y_P743O21 = 0.0
y_P743O23 = 0.0
y_P743O25 = 1.0
y_P743O27 = 0.0
y_P743O3 = 0.0
y_P743O30 = 0.0
y_P743O33 = 0.0
y_P743O7 = 0.0
y_P743O8 = 0.0
y_P744O1 = 0.0
y_P744O11 = 0.0
y_P744O14 = 0.0
y_P744O16 = 0.0
y_P744O18 = 0.0
y_P744O21 = 0.0
y_P744O23 = 0.0
y_P744O25 = 1.0
y_P744O27 = 0.0
y_P744O3 = 0.0
y_P744O30 = 0.0
y_P744O33 = 0.0
y_P744O7 = 0.0
y_P744O8 = 0.0
y_P745O1 = 0.0
y_P745O11 = 0.0
y_P745O14 = 0.0
y_P745O16 = 0.0
y_P745O18 = 0.0
y_P745O21 = 0.0
y_P745O23 = 0.0
y_P745O25 = 1.0
y_P745O27 = 0.0
y_P745O3 = 0.0
y_P745O30 = 0.0
y_P745O33 = 0.0
y_P745O7 = 0.0
y_P745O8 = 0.0
y_P746O1 = 1.0
y_P746O11 = 0.0
y_P746O14 = 0.0
y_P746O16 = 0.0
y_P746O18 = 0.0
y_P746O21 = 0.0
y_P746O23 = 0.0
y_P746O25 = 0.0
y_P746O27 = 0.0
y_P746O3 = 0.0
y_P746O30 = 0.0
y_P746O33 = 0.0
y_P746O7 = 0.0
y_P746O8 = 0.0
y_P747O1 = 0.0
y_P747O11 = 1.0
y_P747O14 = 

y_P797O31 = 0.0
y_P797O32 = 0.0
y_P797O33 = 0.0
y_P797O37 = 0.0
y_P797O4 = 0.0
y_P797O6 = 0.0
y_P797O7 = 0.0
y_P797O8 = 0.0
y_P798O16 = 0.0
y_P798O2 = 1.0
y_P798O23 = 0.0
y_P798O27 = 0.0
y_P798O30 = 0.0
y_P798O31 = 0.0
y_P798O33 = 0.0
y_P798O4 = 0.0
y_P798O7 = 0.0
y_P799O16 = 0.0
y_P799O2 = 1.0
y_P799O23 = 0.0
y_P799O27 = 0.0
y_P799O30 = 0.0
y_P799O31 = 0.0
y_P799O33 = 0.0
y_P799O4 = 0.0
y_P799O7 = 0.0
y_P79O10 = 0.0
y_P79O13 = 0.0
y_P79O19 = 1.0
y_P7O10 = 1.0
y_P800O16 = 0.0
y_P800O2 = 1.0
y_P800O23 = 0.0
y_P800O27 = 0.0
y_P800O30 = 0.0
y_P800O31 = 0.0
y_P800O33 = 0.0
y_P800O4 = 0.0
y_P800O7 = 0.0
y_P801O16 = 0.0
y_P801O2 = 1.0
y_P801O23 = 0.0
y_P801O27 = 0.0
y_P801O30 = 0.0
y_P801O31 = 0.0
y_P801O33 = 0.0
y_P801O4 = 0.0
y_P801O7 = 0.0
y_P802O16 = 0.0
y_P802O2 = 1.0
y_P802O23 = 0.0
y_P802O27 = 0.0
y_P802O30 = 0.0
y_P802O31 = 0.0
y_P802O33 = 0.0
y_P802O4 = 0.0
y_P802O7 = 0.0
y_P803O16 = 0.0
y_P803O2 = 1.0
y_P803O23 = 0.0
y_P803O27 = 0.0
y_P803O30 = 0.0
y_P803O31 = 0.0
y_P803O33 = 0.0
y

y_P859O18 = 0.0
y_P859O21 = 0.0
y_P859O23 = 0.0
y_P859O25 = 0.0
y_P859O27 = 1.0
y_P859O3 = 0.0
y_P859O30 = 0.0
y_P859O33 = 0.0
y_P859O7 = 0.0
y_P859O8 = 0.0
y_P85O10 = 1.0
y_P860O1 = 0.0
y_P860O11 = 0.0
y_P860O14 = 1.0
y_P860O16 = 0.0
y_P860O18 = 0.0
y_P860O21 = 0.0
y_P860O23 = 0.0
y_P860O25 = 0.0
y_P860O27 = 0.0
y_P860O3 = 0.0
y_P860O30 = 0.0
y_P860O33 = 0.0
y_P860O7 = 0.0
y_P860O8 = 0.0
y_P861O1 = 0.0
y_P861O11 = 0.0
y_P861O14 = 0.0
y_P861O16 = 0.0
y_P861O18 = 0.0
y_P861O21 = 0.0
y_P861O23 = 0.0
y_P861O25 = 0.0
y_P861O27 = 0.0
y_P861O3 = 0.0
y_P861O30 = 0.0
y_P861O33 = 0.0
y_P861O7 = 0.0
y_P861O8 = 1.0
y_P862O1 = 0.0
y_P862O11 = 0.0
y_P862O14 = 0.0
y_P862O16 = 0.0
y_P862O18 = 0.0
y_P862O21 = 0.0
y_P862O23 = 0.0
y_P862O25 = 1.0
y_P862O27 = 0.0
y_P862O3 = 0.0
y_P862O30 = 0.0
y_P862O33 = 0.0
y_P862O7 = 0.0
y_P862O8 = 0.0
y_P863O1 = 0.0
y_P863O11 = 0.0
y_P863O14 = 0.0
y_P863O16 = 0.0
y_P863O18 = 0.0
y_P863O21 = 0.0
y_P863O23 = 0.0
y_P863O25 = 0.0
y_P863O27 = 0.0
y_P863O3 = 0.0
y_P863O30 

y_P916O35 = 0.0
y_P916O37 = 0.0
y_P916O38 = 0.0
y_P916O39 = 0.0
y_P916O4 = 0.0
y_P916O41 = 0.0
y_P916O5 = 0.0
y_P916O6 = 0.0
y_P916O7 = 0.0
y_P917O0 = 0.0
y_P917O16 = 0.0
y_P917O22 = 0.0
y_P917O23 = 0.0
y_P917O24 = 0.0
y_P917O27 = 0.0
y_P917O30 = 0.0
y_P917O32 = 0.0
y_P917O33 = 0.0
y_P917O34 = 0.0
y_P917O35 = 0.0
y_P917O37 = 0.0
y_P917O38 = 1.0
y_P917O39 = 0.0
y_P917O4 = 0.0
y_P917O41 = 0.0
y_P917O5 = 0.0
y_P917O6 = 0.0
y_P917O7 = 0.0
y_P918O0 = 0.0
y_P918O16 = 0.0
y_P918O22 = 0.0
y_P918O23 = 0.0
y_P918O24 = 0.0
y_P918O27 = 0.0
y_P918O30 = 0.0
y_P918O32 = 0.0
y_P918O33 = 0.0
y_P918O34 = 0.0
y_P918O35 = 0.0
y_P918O37 = 0.0
y_P918O38 = 0.0
y_P918O39 = 0.0
y_P918O4 = 0.0
y_P918O41 = 0.0
y_P918O5 = 1.0
y_P918O6 = 0.0
y_P918O7 = 0.0
y_P919O0 = 0.0
y_P919O16 = 0.0
y_P919O22 = 0.0
y_P919O23 = 1.0
y_P919O24 = 0.0
y_P919O27 = 0.0
y_P919O30 = 0.0
y_P919O32 = 0.0
y_P919O33 = 0.0
y_P919O34 = 0.0
y_P919O35 = 0.0
y_P919O37 = 0.0
y_P919O38 = 0.0
y_P919O39 = 0.0
y_P919O4 = 0.0
y_P919O41 = 0.0
y_P919O5

y_P966O32 = 0.0
y_P966O33 = 0.0
y_P966O34 = 0.0
y_P966O35 = 0.0
y_P966O37 = 0.0
y_P966O38 = 0.0
y_P966O39 = 0.0
y_P966O4 = 0.0
y_P966O41 = 0.0
y_P966O5 = 0.0
y_P966O6 = 0.0
y_P966O7 = 0.0
y_P967O0 = 0.0
y_P967O16 = 0.0
y_P967O22 = 0.0
y_P967O23 = 0.0
y_P967O24 = 0.0
y_P967O27 = 0.0
y_P967O30 = 1.0
y_P967O32 = 0.0
y_P967O33 = 0.0
y_P967O34 = 0.0
y_P967O35 = 0.0
y_P967O37 = 0.0
y_P967O38 = 0.0
y_P967O39 = 0.0
y_P967O4 = 0.0
y_P967O41 = 0.0
y_P967O5 = 0.0
y_P967O6 = 0.0
y_P967O7 = 0.0
y_P968O0 = 0.0
y_P968O16 = 0.0
y_P968O22 = 0.0
y_P968O23 = 0.0
y_P968O24 = 0.0
y_P968O27 = 0.0
y_P968O30 = 0.0
y_P968O32 = 0.0
y_P968O33 = 0.0
y_P968O34 = 0.0
y_P968O35 = 0.0
y_P968O37 = 0.0
y_P968O38 = 1.0
y_P968O39 = 0.0
y_P968O4 = 0.0
y_P968O41 = 0.0
y_P968O5 = 0.0
y_P968O6 = 0.0
y_P968O7 = 0.0
y_P969O0 = 0.0
y_P969O16 = 0.0
y_P969O22 = 0.0
y_P969O23 = 0.0
y_P969O24 = 0.0
y_P969O27 = 1.0
y_P969O30 = 0.0
y_P969O32 = 0.0
y_P969O33 = 0.0
y_P969O34 = 0.0
y_P969O35 = 0.0
y_P969O37 = 0.0
y_P969O38 = 0.0
y_P969O